In [1]:
import torch
import torch.nn as nn
import sys
from torch.utils.data import DataLoader
from datetime import datetime
import numpy as np
import optuna
sys.path.append("C:/Users/yuuta/Documents/fashion")
from model_learning.model_structure.dataset import TripletDataset
from model_learning.model_structure.image_encoder import ImageEncoderV3

c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [3]:
import importlib
import model_learning
importlib.reload(model_learning.model_structure.image_encoder)

<module 'model_learning.model_structure.image_encoder' from 'C:\\Users/yuuta/Documents/fashion\\model_learning\\model_structure\\image_encoder.py'>

In [4]:
dataset = TripletDataset('C:/Users/yuuta/Documents/fashion/model_learning/compatibility/data/triplet_n_pair.csv', limit=70000)

In [5]:
LEARNING_LATE = 1e-5
BATCH_SIZE = 32
EPOCHS = 3

In [6]:
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size]
)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

loss_fn = nn.TripletMarginLoss(margin=20.0, p=2, eps=1e-6) # , reduction='sum'

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (anchor, positive, negative) in enumerate(dataloader):        
        optimizer.zero_grad()

        anchor_vecter = model(anchor.to(device))
        positive_vecter = model(positive.to(device))
        negative_vecter = model(negative.to(device))

        loss = loss_fn(anchor_vecter, positive_vecter, negative_vecter)
        
        # バックプロパゲーション
        loss.backward()
        optimizer.step()

        if batch % 500 == 0:
            loss, current = loss.item() / BATCH_SIZE, batch * BATCH_SIZE
            print(f"\rloss: {loss:>15f}  [{current:>5d}/{size:>5d}]", end='')

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss = 0
    p_d = 0
    n_d = 0
    with torch.no_grad():
        for (anchor, positive, negative) in dataloader:
            anchor_vecter = model(anchor.to(device))
            positive_vecter = model(positive.to(device))
            negative_vecter = model(negative.to(device))

            positive_d = torch.norm(anchor_vecter - positive_vecter, dim=1, keepdim=True)
            negative_d = torch.norm(anchor_vecter - negative_vecter, dim=1, keepdim=True)
            p_d += torch.sum(positive_d)
            n_d += torch.sum(negative_d)
            loss = loss_fn(anchor_vecter, positive_vecter, negative_vecter)

            test_loss += loss.item()
            
    test_loss /= size
    return n_d - p_d

In [8]:

class Objective:
    def __init__(self):
        pass
    def __call__(self, trial):
        # ハイパーパラメータの設定
        params = {
            'embedding_size': trial.suggest_int('k_l', 32, 1000),
            'dropout_prob': trial.suggest_float('dropout_prob', 0, 1),
            'LEARNING_LATE': trial.suggest_float('LEARNING_LATE', 1e-6, 1e-4),
        }

        model = ImageEncoderV3(dropout_prob=params['dropout_prob'],
                                embedding_size=params['embedding_size']).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=params['LEARNING_LATE'])
        
        for _ in range(EPOCHS):
            train_loop(train_dataloader, model, loss_fn, optimizer)
        score = test_loop(test_dataloader, model, loss_fn)
        print()
        return score

In [9]:
print("start")
print(device)
objective = Objective()
study = optuna.create_study(direction='maximize') # 最大化
study.optimize(objective, timeout=7200 * 30)

[I 2023-12-29 15:06:04,283] A new study created in memory with name: no-name-52589eb6-0ea8-46c7-a99d-58584528e876


start
cuda


c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loss:        0.411249  [48000/49000]

[I 2023-12-29 15:28:46,289] Trial 0 finished with value: 758147.5 and parameters: {'k_l': 206, 'dropout_prob': 0.6958098075569734, 'LEARNING_LATE': 9.59507016520054e-06}. Best is trial 0 with value: 758147.5.


...


loss:        0.037781  [48000/49000]

[I 2023-12-31 14:35:02,107] Trial 126 finished with value: 1045813.5 and parameters: {'k_l': 901, 'dropout_prob': 0.9233651444749218, 'LEARNING_LATE': 8.701744474163741e-05}. Best is trial 67 with value: 4762844.0.



loss:        0.000000  [32000/49000]

[W 2023-12-31 14:47:50,372] Trial 127 failed with parameters: {'k_l': 969, 'dropout_prob': 0.882650761789302, 'LEARNING_LATE': 6.776723277712564e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\yuuta\AppData\Local\Temp\ipykernel_14168\1874164383.py", line 18, in __call__
    train_loop(train_dataloader, model, loss_fn, optimizer)
  File "C:\Users\yuuta\AppData\Local\Temp\ipykernel_14168\2378078362.py", line 8, in train_loop
    negative_vecter = model(negative.to(device))
                            ^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
[W 2023-12-31 14:47:50,374] Trial 127 failed with value None.


KeyboardInterrupt: 

In [ ]:
# ベストパラメータを出力
print('params:', study.best_params)